In [1]:
import pickle
import glob
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import random
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

d:\net work\documents\dheeraj v\assessment 2\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\net work\documents\dheeraj v\assessment 2\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\net work\documents\dheeraj v\assessment 2\.venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\net work\documents\dheera

In [2]:
# To run on GPU
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
# Fetch all the files from the image folder
files = glob.glob('images/**')
print(files)
dictval={}
i = 0

# Iterate over every file and try to save data to the dictval
for file in files:
    print(file)
    if "batches.meta" in file:
        # batches.meta contains the data for the label names 
        # and size of the batch
        with open(file,'rb') as fo:
            data = pickle.load(fo, encoding='bytes')
            print(data)
    else:
        with open(file, 'rb') as fo:
            temp = pickle.load(fo, encoding='bytes')
            #print(temp)
            if i == 0:
                dictval['data']= list(temp[b'data'])
                dictval['labels']= list(temp[b'labels'])
            else:
                dictval['data'] = dictval['data'] + list(temp[b'data'])
                dictval['labels'] = dictval['labels'] + list(temp[b'labels'])
            i+=1


['images\\batches.meta', 'images\\data_batch_1', 'images\\data_batch_2', 'images\\data_batch_3', 'images\\data_batch_4', 'images\\data_batch_5']
images\batches.meta
{b'num_cases_per_batch': 10000, b'label_names': [b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck'], b'num_vis': 3072}
images\data_batch_1
images\data_batch_2
images\data_batch_3
images\data_batch_4
images\data_batch_5


In [4]:
# Convert the bytes to the normal string
print(data[b'label_names'])
labels = [x.decode('utf-8') for x in data[b'label_names']] 
print(labels)

[b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [5]:
alldata = dictval['data']
alldatalabels = dictval['labels']
trainingdata = []
def create_training_data():
    def reshapedata(imdata, imlabel):
        print(len(imdata))
        for i  in range(0,len(imdata)):
        #for i  in range(1,5):
            # This data is the in the format of 3072 array elements
            temp = imdata[i]
            #print(temp)
            #print(len(temp))
            
            # To reshape the data
            img = np.reshape(temp, (3, 32,32)).T
            #print(img.shape)
            
            # Convert the numpy array into the RGB format
            img = Image.fromarray(img, 'RGB')
            
            # To see the image without correct orientation
            #plt.imshow(img)
            #plt.show()
            
            # img is in rotated format, so we need to rotate the image
            # to get the original orientation
            img = img.rotate(270)
            
            # Here gray conversion is done: in our application color images are not need because we 
            # can get the same information in the gray image. 
            # Benefit of using gray image : It will reduce the calculations by 3(RGB have 3 channels)
            img  = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
            
            # Just to make sure that every image is 32*32
            img = cv2.resize(img, (32,32))
            
            # To see the image in the correct orientation
            #plt.imshow(img)
            #plt.show()
            
            # Just to verify that every label is int
            if type(imlabel[i]) != type(2):
                continue
                
            # Stored the labels in another variable
            class_num = imlabel[i]
            
            #print(labels[class_num])
            
            # To create training data: 
            # I have appened the image data and the label
            # temp[0]: This is image
            # temp[1]: This is label
            temp  = [img, class_num]
            trainingdata.append(temp)
            #break
    reshapedata(alldata, alldatalabels)
create_training_data()



50000


In [6]:
# This is to make the data shuffled randomly
import random
random.shuffle(trainingdata)

In [7]:
# This is just to check whether every data is 
# append correctly or not.
for sample in trainingdata[:10]:
    print("label = %d" %sample[1])

X = []
Y = []
# This is to store all the images in X
# and all the labels in Y
for features, label in trainingdata:
    X.append(features)
    Y.append(label)
# To reshape informat of tensorflow
X = np.array(X).reshape(-1,32, 32, 1)
print(X.shape)


label = 6
label = 3
label = 7
label = 5
label = 5
label = 5
label = 5
label = 1
label = 2
label = 9
(50000, 32, 32, 1)


In [8]:
#X = pickle.load(open("X.pickle", 'rb'))
#Y = pickle.load(open("Y.pickle", 'rb'))

# to normalize the data. 
X = X/255.0

# 60% Training data
x_train = X[:30000]
y_train = Y[:30000]

# 20% Testing data
x_test = X[30000:40000]
y_test = Y[30000:40000]

# 20% Validation data
x_val = X[40000:50000]
y_val = Y[40000:50000]


In [9]:
# Model initialization 
model = Sequential()
model.add(Conv2D(256, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))

model.add(Conv2D(128, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))

model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))

model.add(Conv2D(32,(3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))

model.add(Flatten())
model.add(Dense(120))
model.add(Activation("relu"))
model.add(Dense(30))
model.add(Activation("relu"))
model.add(Dense(20))
model.add(Activation("relu"))
# Output layers
model.add(Dense(10))
# Here we will get the outputs in probability 
model.add(Activation("softmax"))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
# Model compilation is done
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# Here we are going to train the model
model.fit(x_train, y_train,batch_size=100, validation_data=(x_val, y_val), epochs = 10)
 

Train on 30000 samples, validate on 10000 samples
Epoch 1/10
30000/30000 [==============================] - 31s 1ms/sample - loss: 2.0049 - acc: 0.2634 - val_loss: 1.7716 - val_acc: 0.3694
Epoch 2/10
30000/30000 [==============================] - 28s 949us/sample - loss: 1.5715 - acc: 0.4336 - val_loss: 1.4437 - val_acc: 0.4923
Epoch 3/10
30000/30000 [==============================] - 29s 951us/sample - loss: 1.2896 - acc: 0.5369 - val_loss: 1.2715 - val_acc: 0.5569
Epoch 4/10
30000/30000 [==============================] - 28s 941us/sample - loss: 1.0989 - acc: 0.6108 - val_loss: 1.2295 - val_acc: 0.5804
Epoch 5/10
30000/30000 [==============================] - 28s 941us/sample - loss: 0.9171 - acc: 0.6777 - val_loss: 1.2164 - val_acc: 0.5871
Epoch 6/10
30000/30000 [==============================] - 28s 938us/sample - loss: 0.7053 - acc: 0.7532 - val_loss: 1.3722 - val_acc: 0.5761
Epoch 7/10
30000/30000 [==============================] - 28s 937us/sample - loss: 0.5129 - acc: 0.8205 - 

In [11]:
# To find the test accuracy
test_loss, test_acc = model.evaluate(x_test, y_test)

10000/10000 [==============================] - 5s 546us/sample - loss: 2.5740 - acc: 0.5354


In [14]:
model.save('ourmodel.h5')
saved_model = tf.keras.models.load_model('ourmodel.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
y_pred = saved_model.predict(x_test)

In [16]:
print(y_pred[8])

[1.4267510e-05 2.0714776e-06 6.5638151e-06 9.7470377e-08 9.2137447e-03
 3.7651032e-06 4.1262282e-11 9.9074990e-01 2.5082119e-08 9.5169353e-06]


In [17]:
predval = 105
count = 0
for i in range(len(y_pred)):
    if np.argmax(y_pred[i]) == y_test[i]:
        count +=1
accuracy = count/len(y_pred)
print(accuracy)
# print(np.argmax(y_pred[predval]))
# print(y_test[predval])

0.6461
